<a href="https://colab.research.google.com/github/YasithKirihena/Telco-Customer-Churn/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import files
import pandas as pd

uploaded = files.upload()


TypeError: 'NoneType' object is not subscriptable

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')# Load the dataset


df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce') # TotalCharges to numeric
print("\nMissing values per column:")
print(df.isnull().sum())
df.dropna(inplace=True)

# show types correct
print("\nFixed Data Types:")
print(df.dtypes)


# ---(1)Exploratory Data Analysis--- #
sns.set_style("whitegrid")

# churned vs stayed
plt.figure(figsize=(6, 4))
sns.countplot(x='Churn', data=df, palette='pastel')
plt.title('Distribution of Churn (Target Variable)')
plt.show()
print()


# contract type
plt.figure(figsize=(8, 5))
sns.countplot(x='Contract', hue='Churn', data=df, palette='Set2')
plt.title('Churn Rate by Contract Type')
plt.show()
print()

# monthly charges vs churn
plt.figure(figsize=(10, 6))
sns.kdeplot(data=df, x='MonthlyCharges', hue='Churn', fill=True, palette='crest')
plt.title('Distribution of Monthly Charges by Churn Status')
plt.show()
print()

# correlation matrix
plt.figure(figsize=(8, 6))
numeric_df = df.select_dtypes(include=['float64', 'int64'])
sns.heatmap(numeric_df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')
plt.show()


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# ---(2.1)PREPROCESSING--- #

df.drop('customerID', axis=1, inplace=True) #customerID not needed

# Binary Columns
le = LabelEncoder()
binary_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']
for col in binary_cols:
    df[col] = le.fit_transform(df[col])

# Categorial Columns
df = pd.get_dummies(df, drop_first=True) #dummy columns

X = df.drop('Churn', axis=1)
y = df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # 80% Training / 20% Testing

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Training Data Shape:", X_train.shape)
print("Testing Data Shape:", X_test.shape)

Training Data Shape: (5625, 30)
Testing Data Shape: (1407, 30)
